# Productividad VTR

In [ ]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta  # Asegúrate de importar timedelta

In [ ]:
import pandas as pd

# Ruta del archivo
ruta = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\soportepostventa_Attendants_Status_2025-07-01_2025-07-30.xlsx"

# Leer el archivo Excel (por defecto lee la primera hoja)
prod = pd.read_excel(ruta)

In [ ]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
prod['MAIL'] = prod['AgentIdentity'].str.split('@').str[0]

In [ ]:
# Lista de columnas a convertir
cols_to_int = [
    "Online", "Invisible", "Pause", "Offline",
    "OpenTicketOnline", "OpenTicketInvisible", "WithoutTicketOnline"
]

# Reemplazar nulos por 0 y convertir a entero
for col in cols_to_int:
    prod[col] = prod[col].fillna(0).astype(int)


In [ ]:
# Convertir el campo a datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Crear campo Fecha en formato dd-mm-aaaa
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%d-%m-%Y")

# Crear campo INTERVALO con hora en formato HH:MM
prod["INTERVALO"] = prod["DateTimeRef"].dt.strftime("%H:%M")


In [ ]:
prod.sample(1)

In [ ]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [ ]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\VTR\Dotación_VTR_20250725.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacion = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [ ]:
dotacion_filtrada = dotacion[
    (dotacion['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacion['PLATAFORMA'].str.lower().isin(['0 wsp. oopp', '0 wsp postventa'])) &
    (dotacion['CARGO'].str.lower() == 'asesor') &
    (dotacion['ESTADO_ASESOR'].str.lower().isin(['activo', 'licencia', 'vacaciones']))
]


In [ ]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtrada['MAIL'] = dotacion_filtrada['CORREO_ELECTRONICO'].str.split('@').str[0]

In [ ]:
# Creamos versiones en minúscula del campo MAIL para el join
prod['MAIL'] = prod['MAIL'].str.lower()
dotacion_filtrada['MAIL'] = dotacion_filtrada['MAIL'].str.lower()
blip_ha['MAIL'] = blip_ha['MAIL'].str.lower()

In [ ]:
# Hacer merge directo por MAIL
prod = prod.merge(
    dotacion_filtrada[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
prod['MARCA'] = prod['EMPRESA_PAIS'].notna().map({True: 'VTR', False: ''})

In [ ]:
# Asegurarnos de que DateTimeRef es datetime
prod["DateTimeRef"] = pd.to_datetime(prod["DateTimeRef"], dayfirst=True, errors="coerce")

# Reconvertir campo Fecha a formato YYYY-MM-DD
prod["Fecha"] = prod["DateTimeRef"].dt.strftime("%Y-%m-%d")

In [ ]:
# --- 3) Contar SequentialId por MAIL-Fecha-INTERVALO en blip_ha ---
blip_counts = (
    blip_ha
    .groupby(["MAIL", "Fecha", "INTERVALO"], as_index=False)["SequentialId"]
    .count()
    .rename(columns={"SequentialId": "Atenciones"})
)

# --- 4) Merge a prod para traer Atenciones (dejar 0 si no hay match) ---
prod = prod.merge(
    blip_counts,
    on=["MAIL", "Fecha", "INTERVALO"],
    how="left"
)

prod["Atenciones"] = prod["Atenciones"].fillna(0).astype(int)

In [ ]:
prod.sample(4)

In [ ]:
# Opcional si Online pudiese venir como texto
prod['H-Productiva'] = (prod['Online'] >= 10).astype(int)

In [ ]:
# Eliminar filas con NaN en EMPRESA_PAIS
prod = prod[prod['EMPRESA_PAIS'].notna()].copy()

In [ ]:
# Ahora sí, guardás el archivo
prod.to_excel(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Productividad_JUL.xlsx", index=False)

# Productividad CLARO

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta  # Asegúrate de importar timedelta

In [2]:
import pandas as pd

# Ruta del archivo
ruta = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\soporteposventahogarclara_Attendants_Status_2025-08-01_2025-08-18.xlsx"

# Leer el archivo Excel (por defecto lee la primera hoja)
prodc = pd.read_excel(ruta)

In [3]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
prodc['MAIL'] = prodc['AgentIdentity'].str.split('@').str[0]

In [4]:
# Lista de columnas a convertir
cols_to_int = [
    "Online", "Invisible", "Pause", "Offline",
    "OpenTicketOnline", "OpenTicketInvisible", "WithoutTicketOnline"
]

# Reemplazar nulos por 0 y convertir a entero
for col in cols_to_int:
    prodc[col] = prodc[col].fillna(0).astype(int)


In [5]:
# Convertir el campo a datetime
prodc["DateTimeRef"] = pd.to_datetime(prodc["DateTimeRef"], dayfirst=True, errors="coerce")

# Crear campo Fecha en formato dd-mm-aaaa
prodc["Fecha"] = prodc["DateTimeRef"].dt.strftime("%d-%m-%Y")

# Crear campo INTERVALO con hora en formato HH:MM
prodc["INTERVALO"] = prodc["DateTimeRef"].dt.strftime("%H:%M")


In [6]:
# Definir la ruta del archivo correctamente
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Blip Power Bi Base\f_blip_ha.csv"
blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'

# Ajustar las opciones para mostrar todas las columnas
pd.set_option('display.max_columns', None)

C:\Users\wduran\AppData\Local\Temp\ipykernel_26912\3964556920.py:3: DtypeWarning: Columns (17,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,101) have mixed types. Specify dtype option on import or set low_memory=False.
  blip_ha = pd.read_csv(archivo, delimiter=',', encoding='latin1')  # Prueba con 'latin1'


In [7]:
# Define the file path
archivo = r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Dotaciones\CLARO\Dotación_CLARO_20250815.xlsx"
# Read the specific sheet named 'Dotacion' from the Excel file
dotacionc = pd.read_excel(archivo, sheet_name='Dotacion')  # Specify the sheet name

In [8]:
dotacionc.sample(1)

,CEDULA,RUT_RAC,NOMBRE_ASESOR,CARGO,CLARO_MAS,ESAC,ONE,SUR,TIPIFICA,ID_ANDES,ID_GENESYS,USUARIO_RED,JORNADA,CORREO_ELECTRONICO,INGRESO_A_PLATAFORMA,EGRESO_DE_PLATAFORMA,ESTADO_CONTRATO,ESTADO_ASESOR,EMPRESA_PAIS,PLATAFORMA,SUPERVISOR,COORDINADOR,JEFE,HABILIDAD,CRM_CAPACITADO,CROSSSELLING,ESTADO_POSICION,OBSERVACION,SITE
648,1111744321,0057300894-4,PARRA CARDENAS PAULA MARYURI,ASESOR,NaN,NaN,NaN,NaN,NaN,PPARRACA,PPARRACA,NaN,44,paula.parracardenas@ext.clarovtr.cl,2025-01-29,NaT,VIGENTE,ACTIVO,ATENTO COLOMBIA,CONFIRMACION VOZ,MURILLO RICO ANGELA PIEDAD,SALAZAR GRISALES ERIKA,RAMIREZ RESTREPO YARIMA,TEC-CONVTA,ANDES,NO,HOME,NaN,PEREIRA


In [9]:
dotacion_filtradac = dotacionc[
    (dotacionc['ESTADO_CONTRATO'].str.lower() == 'vigente') &
    (dotacionc['PLATAFORMA'].str.lower().isin(['postventa no voz'])) &
    (dotacionc['CARGO'].str.lower() == 'asesor') &
    (dotacionc['ESTADO_ASESOR'].str.lower().isin(['activo']))
]

In [10]:
# Crear la nueva columna MAIL que extrae la parte izquierda del arroba
dotacion_filtradac['MAIL'] = dotacion_filtradac['CORREO_ELECTRONICO'].str.split('@').str[0]

C:\Users\wduran\AppData\Local\Temp\ipykernel_26912\4167450716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtradac['MAIL'] = dotacion_filtradac['CORREO_ELECTRONICO'].str.split('@').str[0]


In [11]:
# Creamos versiones en minúscula del campo MAIL para el join
prodc['MAIL'] = prodc['MAIL'].str.lower()
dotacion_filtradac['MAIL'] = dotacion_filtradac['MAIL'].str.lower()
blip_ha['MAIL'] = blip_ha['MAIL'].str.lower()

C:\Users\wduran\AppData\Local\Temp\ipykernel_26912\1817360879.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dotacion_filtradac['MAIL'] = dotacion_filtradac['MAIL'].str.lower()


In [12]:
# Hacer merge directo por MAIL
prodc = prodc.merge(
    dotacion_filtradac[['MAIL', 'EMPRESA_PAIS']], 
    on='MAIL', 
    how='left'
)

# Crear campo MARCA: 'VTR' si coincide, vacío si no
prodc['MARCA'] = prodc['EMPRESA_PAIS'].notna().map({True: 'CLARO', False: ''})

In [13]:
# Asegurarnos de que DateTimeRef es datetime
prodc["DateTimeRef"] = pd.to_datetime(prodc["DateTimeRef"], dayfirst=True, errors="coerce")

# Reconvertir campo Fecha a formato YYYY-MM-DD
prodc["Fecha"] = prodc["DateTimeRef"].dt.strftime("%Y-%m-%d")

In [14]:
# --- 3) Contar SequentialId por MAIL-Fecha-INTERVALO en blip_ha ---
blip_counts = (
    blip_ha
    .groupby(["MAIL", "Fecha", "INTERVALO"], as_index=False)["SequentialId"]
    .count()
    .rename(columns={"SequentialId": "Atenciones"})
)

# --- 4) Merge a prod para traer Atenciones (dejar 0 si no hay match) ---
prodc = prodc.merge(
    blip_counts,
    on=["MAIL", "Fecha", "INTERVALO"],
    how="left"
)

prodc["Atenciones"] = prodc["Atenciones"].fillna(0).astype(int)

In [15]:
# Opcional si Online pudiese venir como texto
prodc['H-Productiva'] = (prodc['Online'] >= 10).astype(int)

In [16]:
# Eliminar filas con NaN en EMPRESA_PAIS
prodc = prodc[prodc['EMPRESA_PAIS'].notna()].copy()

In [17]:
prodc

,AgentIdentity,BotId,Teams,Device,DateTimeRef,Online,Invisible,Pause,Offline,OpenTicketOnline,OpenTicketInvisible,WithoutTicketOnline,WithoutTicketInvisible,MAIL,Fecha,INTERVALO,EMPRESA_PAIS,MARCA,Atenciones,H-Productiva
0,alexis.mondragonrequejo@e.vtr.cl,soporteposventahogarclara@msging.net,"[Babysitting Hogar - Dia 14, Babysitting Hogar...",Web,2025-08-01 00:00:00,53,6,0,0,34,0,19,6.43,alexis.mondragonrequejo,2025-08-01,00:00,KONECTA PERU,CLARO,0,1
1,alexis.mondragonrequejo@e.vtr.cl,soporteposventahogarclara@msging.net,"[Babysitting Hogar - Dia 14, Babysitting Hogar...",Web,2025-08-01 01:00:00,59,0,0,0,1,0,58,0.00,alexis.mondragonrequejo,2025-08-01,01:00,KONECTA PERU,CLARO,0,1
2,alexis.mondragonrequejo@e.vtr.cl,soporteposventahogarclara@msging.net,"[Babysitting Hogar - Dia 14, Babysitting Hogar...",Web,2025-08-01 02:00:00,59,0,0,0,0,0,59,0.00,alexis.mondragonrequejo,2025-08-01,02:00,KONECTA PERU,CLARO,0,1
3,alexis.mondragonrequejo@e.vtr.cl,soporteposventahogarclara@msging.net,"[Babysitting Hogar - Dia 14, Babysitting Hogar...",Web,2025-08-01 03:00:00,59,0,0,0,0,0,59,0.00,alexis.mondragonrequejo,2025-08-01,03:00,KONECTA PERU,CLARO,0,1
4,alexis.mondragonrequejo@e.vtr.cl,soporteposventahogarclara@msging.net,"[Babysitting Hogar - Dia 14, Babysitting Hogar...",Web,2025-08-01 04:00:00,59,0,0,0,0,0,59,0.00,alexis.mondragonrequejo,2025-08-01,04:00,KONECTA PERU,CLARO,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,valentin.rodriguezsulca@ext.clarochile.cl,soporteposventahogarclara@msging.net,"[VideoLlamada];[Babysitting Hogar - Dia 14, Ba...",Web,2025-08-10 19:00:00,0,0,0,59,0,0,0,0.00,valentin.rodriguezsulca,2025-08-10,19:00,KONECTA PERU,CLARO,0,0
9356,valentin.rodriguezsulca@ext.clarochile.cl,soporteposventahogarclara@msging.net,"[VideoLlamada];[Babysitting Hogar - Dia 14, Ba...",Web,2025-08-10 20:00:00,0,0,0,59,0,0,0,0.00,valentin.rodriguezsulca,2025-08-10,20:00,KONECTA PERU,CLARO,0,0
9357,valentin.rodriguezsulca@ext.clarochile.cl,soporteposventahogarclara@msging.net,"[VideoLlamada];[Babysitting Hogar - Dia 14, Ba...",Web,2025-08-10 21:00:00,0,0,0,59,0,0,0,0.00,valentin.rodriguezsulca,2025-08-10,21:00,KONECTA PERU,CLARO,0,0
9358,valentin.rodriguezsulca@ext.clarochile.cl,soporteposventahogarclara@msging.net,"[VideoLlamada];[Babysitting Hogar - Dia 14, Ba...",Web,2025-08-10 22:00:00,0,0,0,59,0,0,0,0.00,valentin.rodriguezsulca,2025-08-10,22:00,KONECTA PERU,CLARO,0,0


In [18]:
# Ahora sí, guardás el archivo
prodc.to_excel(r"C:\Users\wduran\OneDrive - CLAROCHILE\Digital Team\Productividad\Productividad_CLARO_AGO.xlsx", index=False)